In [ ]:
from common_imports import *  # Importeren van get_engine, show_home_button, Path, logging, pd, text, datetime, widgets, etc.
show_home_button()  # Gedefinieerd in common_imports.py
engine = get_engine(autocommit=True)  # autocommit parameter ondersteund door db_connection.py

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(levelname)-8s | %(message)s")

class DatabaseManager:
    def __init__(self, engine):
        self.engine = engine

    def _get_method_id(self, minutes: int) -> int:
        q = text("""
            SELECT Id
            FROM dbo.TBL_Ref_Register_StorageMethod
            WHERE CollectInterval = :min;
        """)
        df = pd.read_sql(q, self.engine, params={"min": minutes})
        if df.empty or pd.isnull(df.loc[0, "Id"]):
            raise ValueError(f"Geen opslagmethode (StorageMethodId) gevonden voor interval {minutes} minuten.")
        return int(df.loc[0, "Id"])

    def _get_utc_time(self, local_time_dt: datetime) -> datetime:
        q = text("""
            SELECT TOP 1 UTC
            FROM dbo.TBL_LocalTime
            WHERE LocalTime <= :local_time
            ORDER BY UTC DESC;
        """)
        df = pd.read_sql(q, self.engine, params={"local_time": local_time_dt})
        if df.empty:
            raise ValueError(f"Kon UTC tijd niet bepalen voor lokale tijd {local_time_dt.strftime('%Y-%m-%d %H:%M:%S')}")
        utc_time_result = df.loc[0, "UTC"]
        if isinstance(utc_time_result, str):
             return datetime.fromisoformat(utc_time_result)
        return utc_time_result

    def _check_sp_exists(self, sp_name: str) -> bool:
        q_check_sp = text("SELECT OBJECT_ID(:sp_name, 'P') AS sp_object_id;")
        df_check_sp = pd.read_sql(q_check_sp, self.engine, params={"sp_name": sp_name})
        return not df_check_sp.empty and df_check_sp.loc[0, "sp_object_id"] is not None

    def fetch_register_ids(self, registrator_id: int) -> list[int]:
        q = text("""
            SELECT ID AS RegisterId
            FROM dbo.TBL_Register
            WHERE RegistratorId = :rid
            ORDER BY ID;
        """)
        df = pd.read_sql(q, self.engine, params={"rid": registrator_id})
        return df["RegisterId"].tolist()

    def fetch_register_settings(self, registrator_id: int) -> pd.DataFrame:
        q = text("""
            SELECT ID AS RegisterId, CollectInterval, StorageMethodId
            FROM dbo.TBL_Register
            WHERE RegistratorId = :rid
            ORDER BY ID;
        """)
        return pd.read_sql(q, self.engine, params={"rid": registrator_id})

    def update_registers(self, registrator_id: int, minutes: int):
        method_id = self._get_method_id(minutes)
        sql = text("""
            UPDATE dbo.TBL_Register
            SET CollectInterval = :minutes, StorageMethodId = :method
            WHERE RegistratorId = :rid;
        """)
        with self.engine.connect() as conn:
            conn.execute(sql, {"minutes": minutes, "method": method_id, "rid": registrator_id})
        logging.info("Registers voor RegistratorId %d zijn bijgewerkt naar CollectInterval=%d, StorageMethodId=%d.", registrator_id, minutes, method_id)

    def run_repair(self, registrator_id: int, start_local_for_repair_str: str,
                   fixed_end_local_for_repair_str: str, do_update_for_repair: bool):

        repair_log_entries = []

        # === 1. Validatie en conversies ===
        try:
            input_start_local_dt = datetime.strptime(start_local_for_repair_str, '%Y-%m-%d %H:%M')
            fixed_end_local_dt = datetime.strptime(fixed_end_local_for_repair_str, '%Y-%m-%d %H:%M:%S')
        except ValueError as e:
            logging.error(f"Ongeldig datumformaat: {e}")
            raise ValueError(f"Ongeldig datumformaat: {e}")

        # Check of registrator bestaat
        q_check_reg = text("SELECT 1 FROM dbo.TBL_Registrator WHERE Id = :rid")
        if pd.read_sql(q_check_reg, self.engine, params={"rid": registrator_id}).empty:
            msg = f"RegistratorId {registrator_id} bestaat niet."
            logging.error(msg)
            raise ValueError(msg)

        # Bepaal UTC tijden
        utc_start_repair = self._get_utc_time(input_start_local_dt)
        utc_end_repair = self._get_utc_time(fixed_end_local_dt)

        # Verzamel Register IDs (identiek aan SQL - ALTIJD alles van RegistratorId)
        register_ids_for_repair = self.fetch_register_ids(registrator_id)

        if not register_ids_for_repair:
            logging.info(f"Geen registers gevonden voor RegistratorId {registrator_id} om te repareren. Reparatie overgeslagen.")
            df_log = pd.DataFrame(repair_log_entries, columns=['RegisterId', 'Action', 'Message', 'LogDateTime'])
            df_pivot = pd.DataFrame({'utcperiod': pd.Series(dtype='datetime64[ns]')})
            return df_log, df_pivot

        # === 2. Loop over alle registers, identiek aan cursor in SQL ===
        with self.engine.connect() as conn:
            for current_register_id in register_ids_for_repair:
                now_dt = datetime.now()
                repair_log_entries.append({
                    "RegisterId": current_register_id, "Action": "StartFix",
                    "Message": "Aanroepen dbo.FixRegisterStorage", "LogDateTime": now_dt
                })

                if self._check_sp_exists('dbo.FixRegisterStorage'):
                    try:
                        sp_sql = text("EXEC dbo.FixRegisterStorage @registerid = :reg_id, @utcstart = :utc_s, @utcend = :utc_e")
                        conn.execute(sp_sql, {
                            "reg_id": current_register_id,
                            "utc_s": utc_start_repair,
                            "utc_e": utc_end_repair
                        })
                    except Exception as e_sp:
                        repair_log_entries.append({
                            "RegisterId": current_register_id, "Action": "ErrorFix",
                            "Message": f"Fout bij aanroepen dbo.FixRegisterStorage: {str(e_sp)}", "LogDateTime": datetime.now()
                        })
                else:
                    repair_log_entries.append({
                        "RegisterId": current_register_id, "Action": "ErrorFix",
                        "Message": "Stored procedure dbo.FixRegisterStorage niet gevonden.", "LogDateTime": datetime.now()
                    })

                repair_log_entries.append({
                    "RegisterId": current_register_id, "Action": "EndFix",
                    "Message": "dbo.FixRegisterStorage voltooid", "LogDateTime": datetime.now()
                })

                if do_update_for_repair:
                    repair_log_entries.append({
                        "RegisterId": current_register_id, "Action": "StartUpdate",
                        "Message": "Data bijwerken...", "LogDateTime": datetime.now()
                    })

                    params_update = {
                        "reg_id": current_register_id,
                        "utc_s": utc_start_repair,
                        "utc_e": utc_end_repair
                    }

                    del_repaired_sql = text("""
                        DELETE FROM [eds2_work_archive].dbo.TBL_Data_Repaired 
                        WHERE registerid = :reg_id AND utcperiod BETWEEN :utc_s AND :utc_e;
                    """)
                    conn.execute(del_repaired_sql, params_update)

                    ins_repaired_sql = text("""
                        INSERT INTO [eds2_archive].dbo.TBL_Data_Repaired(registerid, utcperiod, period, consumption, statusid) 
                        SELECT registerid, utcperiod, period, consumption, statusid FROM dbo.TBL_data 
                        WHERE registerid = :reg_id AND utcperiod BETWEEN :utc_s AND :utc_e;
                    """)
                    conn.execute(ins_repaired_sql, params_update)

                    del_data_sql = text("""
                        DELETE FROM dbo.TBL_data 
                        WHERE registerid = :reg_id AND utcperiod BETWEEN :utc_s AND :utc_e;
                    """)
                    conn.execute(del_data_sql, params_update)

                    ins_data_sql = text("""
                        INSERT INTO dbo.TBL_data(registerid, utcperiod, period, consumption, statusid) 
                        SELECT registerid, utcperiod, period, consumption, statusid FROM dbo.TBL_data_kladblok 
                        WHERE registerid = :reg_id AND utcperiod BETWEEN :utc_s AND :utc_e;
                    """)
                    conn.execute(ins_data_sql, params_update)

                    repair_log_entries.append({
                        "RegisterId": current_register_id, "Action": "EndUpdate",
                        "Message": "Data bijwerken voltooid.", "LogDateTime": datetime.now()
                    })
                else:
                    repair_log_entries.append({
                        "RegisterId": current_register_id, "Action": "SkipUpdate",
                        "Message": "Data bijwerken overgeslagen (DoUpdate=0).", "LogDateTime": datetime.now()
                    })

        # === 3. Zet log in DataFrame en sorteer ===
        df_log = pd.DataFrame(repair_log_entries)
        if not df_log.empty:
            df_log = df_log.sort_values(by=['RegisterId', 'LogDateTime'])

        # === 4. Dynamische PIVOT ===
        if not register_ids_for_repair:
            logging.info("Geen register IDs gevonden voor de PIVOT query.")
            df_pivot = pd.DataFrame({'utcperiod': pd.Series(dtype='datetime64[ns]')})
            return df_log, df_pivot

        # Beschrijving ophalen zoals in SQL
        ph_pivot_desc = ",".join([f":id{i}" for i in range(len(register_ids_for_repair))])
        params_pivot_desc = {f"id{i}": v for i, v in enumerate(register_ids_for_repair)}
        q_desc = text(f"""
            SELECT ID, Description 
            FROM dbo.TBL_Register 
            WHERE ID IN ({ph_pivot_desc})
        """)
        df_desc_data = pd.read_sql(q_desc, self.engine, params=params_pivot_desc)
        desc_map = pd.Series(df_desc_data.Description.values, index=df_desc_data.ID).to_dict()

        pivot_select_cols_list = []
        for rid in register_ids_for_repair:
            description = desc_map.get(rid)
            col_alias = description if pd.notna(description) and description else f"Register {rid}"
            pivot_select_cols_list.append(f"[{rid}] AS [{col_alias} ({rid})]")

        pivot_select_cols_str = ",".join(pivot_select_cols_list)
        pivot_ids_str = ",".join(map(str, register_ids_for_repair))
        pivot_cols_str = ",".join([f"[{rid}]" for rid in register_ids_for_repair])

        if not pivot_select_cols_str:
            logging.info("Kon geen kolommen genereren voor de PIVOT query.")
            df_pivot = pd.DataFrame({'utcperiod': pd.Series(dtype='datetime64[ns]')})
            return df_log, df_pivot

        pivot_sql = text(f"""
            SELECT utcperiod, {pivot_select_cols_str}
            FROM (
                SELECT utcperiod, registerid, consumption
                FROM dbo.TBL_data
                WHERE registerid IN ({pivot_ids_str})
                  AND utcperiod BETWEEN :ParamUtcStart AND :ParamUtcEnd
            ) AS SourceTable
            PIVOT (
                MAX(consumption)
                FOR registerid IN ({pivot_cols_str})
            ) AS PivotTable
            ORDER BY utcperiod;
        """)
        df_pivot = pd.read_sql_query(pivot_sql, self.engine, params={
            "ParamUtcStart": utc_start_repair,
            "ParamUtcEnd": utc_end_repair
        })

        return df_log, df_pivot

class UIManager:
    def __init__(self, db: DatabaseManager):
        self.db = db
        self.fixed_end = "2026-01-01 00:00:00"
        self._build_ui()

    def _build_ui(self):
        self.out = widgets.Output()
        self.reg_input_label = widgets.Label("RegistratorId:", layout=widgets.Layout(display='flex', align_items='center'))
        self.reg_input = widgets.Text(description='', placeholder='Vul RegistratorId in', value='97798', layout=widgets.Layout(width='100px'))
        self.fetch_btn = widgets.Button(description='Fetch RegisterIDs', icon='database', tooltip='Fetch register IDs and their current settings')
        self.run_btn = widgets.Button(description='Run Repair', icon='play', disabled=True, tooltip='Run the repair process with the specified settings')
        self.run_btn.add_class('mod-success')
        toolbar_row = widgets.HBox([self.reg_input_label, self.reg_input, self.fetch_btn, self.run_btn], layout=widgets.Layout(align_items='center', gap='8px'))

        self.start_local_label = widgets.Label("StartLocal:", layout=widgets.Layout(display='flex', align_items='center'))
        self.start_local = widgets.Text(value="2024-01-01 00:05", description='', layout=widgets.Layout(width='180px'))
        self.end_local_html = widgets.HTML(f"<b>EndLocal (Fixed):</b> {self.fixed_end}", layout=widgets.Layout(display='flex', align_items='center'))

        self.minutes_label = widgets.Label("Interval (min):", layout=widgets.Layout(display='flex', align_items='center'))
        self.interval_dropdown = widgets.Dropdown(options=[("5 minuten",5), ("15 minuten",15)], value=5, description='', layout=widgets.Layout(width='130px'))

        self.do_update = widgets.Checkbox(value=True, description='DoUpdate', indent=False, layout=widgets.Layout(display='flex', align_items='center'))

        form_fields_row = widgets.HBox(
            [self.start_local_label, self.start_local, self.end_local_html, self.minutes_label, self.interval_dropdown, self.do_update],
            layout=widgets.Layout(align_items='center', flex_wrap='nowrap', gap='8px')
        )
        self.settings_df_title = widgets.HTML("<b>Current settings:</b>")
        self.settings_df = widgets.Output(layout=widgets.Layout(width='100%'))
        controls_box = widgets.VBox(
            [toolbar_row, form_fields_row, self.settings_df_title, self.settings_df],
            layout=widgets.Layout(padding='8px', border='1px solid #cccccc', border_radius='4px', gap='12px')
        )
        self.fetch_btn.on_click(self.on_fetch)
        self.run_btn.on_click(self.on_run)
        self.tabs = widgets.Tab(layout=widgets.Layout(margin='10px 0 0 0'))
        display(controls_box, self.out, self.tabs)

    def on_fetch(self, _):
        with self.out:
            clear_output(wait=True)
            print(f"Fetching registers for RegistratorId: {self.reg_input.value}...")
        try:
            registrator_id_val_str = self.reg_input.value
            if not registrator_id_val_str:
                with self.out:
                    clear_output(wait=True)
                    print("RegistratorId cannot be empty.")
                self.run_btn.disabled = True
                with self.settings_df: clear_output(wait=True)
                return

            registrator_id_val = int(registrator_id_val_str)
            regs = self.db.fetch_register_ids(registrator_id_val)

            with self.out:
                clear_output(wait=True)
                if regs: print(f"Successfully fetched. Register IDs: {', '.join(map(str, regs))}")
                else: print(f"No registers found for RegistratorId: {registrator_id_val}.")

            df = self.db.fetch_register_settings(registrator_id_val)
            with self.settings_df:
                clear_output(wait=True)
                if not df.empty:
                    styles = [{'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', 'auto'), ('border', 'none')]}, {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#f0f0f0'), ('padding', '8px'), ('border-bottom', '1px solid #ddd')]}, {'selector': 'td', 'props': [('text-align', 'left'), ('padding', '8px'), ('border-bottom', '1px solid #eee')]}, {'selector': 'tr:nth-child(even) td', 'props': [('background-color', '#f9f9f9')]}, {'selector': 'tr:nth-child(odd) td', 'props': [('background-color', '#ffffff')]}, {'selector': 'th, td', 'props': [('border-left', 'none'), ('border-right', 'none'), ('border-top', 'none')]}]
                    df_styled = df.style.set_table_styles(styles).hide(axis="index")
                    display(HTML(df_styled.to_html()))
                elif regs: print("Registers found, but no specific settings retrieved.")
            self.run_btn.disabled = not bool(regs)
            self.reg_ids = regs
        except Exception as e:
            with self.out:
                clear_output(wait=True)
                logging.error(f"Error during fetch: {e}")
                print(f"An error occurred: {e}")
            self.run_btn.disabled = True
            with self.settings_df: clear_output(wait=True)

    def on_run(self, _):
        with self.out:
            clear_output(wait=True)
            print(f"Preparing to run repair for RegistratorId: {self.reg_input.value} with interval {self.interval_dropdown.value} minutes.")
        try:
            registrator_id_val = int(self.reg_input.value)
            mins = self.interval_dropdown.value

            # UPDATE altijd op RegistratorId zoals in SQL
            self.db.update_registers(registrator_id_val, mins)
            with self.out:
                print(f"Registers voor RegistratorId {registrator_id_val} zijn bijgewerkt naar interval {mins} minuten.")

            df_log, df_pivot = self.db.run_repair(
                registrator_id_val,
                self.start_local.value,
                self.fixed_end,
                self.do_update.value
            )
            with self.out:
                clear_output(wait=True)
                print("Repair process completed.")
                if self.do_update.value: print("Data update was performed.")
                else: print("Data update was skipped as per selection.")

            self.tabs.children = [widgets.Output(), widgets.Output()]
            self.tabs.set_title(0, 'Repair Log')
            self.tabs.set_title(1, 'Pivot Data')

            with self.tabs.children[0]:
                clear_output(wait=True)
                if not df_log.empty: display(df_log)
                else: print("Repair log is empty.")
            with self.tabs.children[1]:
                clear_output(wait=True)
                if not df_pivot.empty: display(df_pivot)
                else: print("Pivot data is empty.")

            self.on_fetch(None)
        except Exception as e:
            with self.out:
                clear_output(wait=True)
                logging.error(f"Error during run: {e}")
                print(f"An error occurred during the repair process: {e}")

dbm = DatabaseManager(engine)
ui = UIManager(dbm)

ModuleNotFoundError: No module named 'common_imports'